In [1]:

!pip install langchain_openai langchain_community pypdf chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 121.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.6 MB/s eta

In [2]:

import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
import urllib.request
import gradio as gr

In [3]:
# OpenAI API 키 설정 (본인의 API 키로 변경)
from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
API_KEY = os.getenv("OPENAI_API_KEY")

# from google.colab import userdata
# api_key=userdata.get('api_key')
# os.environ["OPENAI_API_KEY"] = api_key


In [4]:
urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf", filename="2020_경제금융용어 700선_게시.pdf")


('2020_경제금융용어 700선_게시.pdf', <http.client.HTTPMessage at 0x12c1b0fa0>)

In [5]:

loader = PyPDFLoader("2020_경제금융용어 700선_게시.pdf")
texts = loader.load_and_split()
print('문서의 수 :', len(texts))

문서의 수 : 366


In [6]:

texts[15]

Document(metadata={'producer': 'Smart PDF Imposition 1.0', 'creator': 'Smart PDF Imposition 1.0', 'creationdate': 'January 31, 2018 16:21', 'author': 'INSUNG DATA INC.', 'creationdate--text': 'January 31, 2018 16:21', 'moddate': 'January 31, 2018 16:21', 'moddate--text': 'January 31, 2018 16:21', 'subject': 'CoreImposition PDF 1.0 Output', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'source': '2020_경제금융용어 700선_게시.pdf', 'total_pages': 371, 'page': 19, 'page_label': '20'}, page_content='3\nㄱ \n신용을 모두 포괄하는 것으로 크게 가계대출과 판매신용으로 구분된다. 현재 가계신용통\n계는 2002년 말 잔액부터 분기별로 제공되고 있다. 가계신용통계는 우리나라 가계부문의 \n부채 규모 및 변동 등을 파악하는 데 널리 활용되고 있다 .\n가계처분가능소득 \n가계처분가능소득(PDI; Personal Disposable Income)은 가계가 맘대로 소비와 저축\n으로 처분할 수 있는 소득을 의미한다. 흔히 국민들의 생활수준을 파악해 볼 수 있는 \n지표로 1인당 GNI가 널리 쓰이고 있으나 국민총소득에는 가계 뿐 아니라 기업 금융기관 \n정부가 벌어 들인 소득이 모두 포함되어 있다. 따라서 기업과 금융기관 등이 가계부문 \n보다 더 많은 소득을 벌어 1인당 국민총소득(GNI)이 높아진 경우에는 가계가 느끼는 \n체감경기는 전체 경기와 괴리가 있게 된다. 1인당 가계총처분가능소득(PGDI; Personal \nGross Disposable Income)은 가계부문의 총처분가능소득을 연앙인구로 나누

In [7]:
print(texts[15].page_content)

3
ㄱ 
신용을 모두 포괄하는 것으로 크게 가계대출과 판매신용으로 구분된다. 현재 가계신용통
계는 2002년 말 잔액부터 분기별로 제공되고 있다. 가계신용통계는 우리나라 가계부문의 
부채 규모 및 변동 등을 파악하는 데 널리 활용되고 있다 .
가계처분가능소득 
가계처분가능소득(PDI; Personal Disposable Income)은 가계가 맘대로 소비와 저축
으로 처분할 수 있는 소득을 의미한다. 흔히 국민들의 생활수준을 파악해 볼 수 있는 
지표로 1인당 GNI가 널리 쓰이고 있으나 국민총소득에는 가계 뿐 아니라 기업 금융기관 
정부가 벌어 들인 소득이 모두 포함되어 있다. 따라서 기업과 금융기관 등이 가계부문 
보다 더 많은 소득을 벌어 1인당 국민총소득(GNI)이 높아진 경우에는 가계가 느끼는 
체감경기는 전체 경기와 괴리가 있게 된다. 1인당 가계총처분가능소득(PGDI; Personal 
Gross Disposable Income)은 가계부문의 총처분가능소득을 연앙인구로 나누어 계산한 
지표로 가계의 구매력을 가장 정확히 가늠해 볼 수 있는 소득지표이다 . 
 연관검색어 : 국민총소득 (GNI)
가교은행
청산 대상 금융기관의 자산, 부채를 임시로 넘겨받아 예금, 출금 등의 업무를 대행하고 
합병, 채권채무관계 조정 등 후속조치를 수행하는 은행을 말한다. 금융기관이 파산한 경우 
예금보험제도는 청산, 매각, 자산부채승계, 가교은행을 통한 인수 등의 수단을 통해 부실 
금융기관을 처리한다. 이 중 가교은행을 통한 인수방식은 파산은행의 처리에 상당한 시간이 
소요될 것으로 예상할 경우 사용되며 새로운 은행을 설립하여 자산, 부채를 포괄승계하도록 
하고, 인수 희망자를 물색하는 등의 조치를 취하게 된다. 가교은행을 이용한 처리방식은 
금융기관 파산에 따른 충격을 완화하고 시간적인 여유를 확보할 수 있는 것이 장점이다. 
임무를 마치면 정리대상 금융기관과 마찬가지로 없어지는 한시적 기관이다.
가동률
생산능력 대비 생산실적의 백분율(생산실적/생산능력×100)로

In [8]:
print(texts[0].page_content)

iii
찾아보기
한국은행은 국민들이 경제 및 금융에 대한 이해도를 높이고 경제에 관한 합리적인 
의사결정 능력을 키울 수 있도록 현장 경제교육, 온라인 경제교육, 경제교육 콘텐츠 
개발 등 대국민 경제교육을 다양하게 수행해 오고 있습니다 . 
이의 일환으로 2018년 
 경제금융용어 700선
 을 발간하였는데 그간 동 책자에 대한 
수요가 꾸준히 늘어남에 따라 이번에 추가로 발간하게 되었습니다 .
지난번 내용과 같이 통화정책, 실물경제, 금융안정, 지급결제 등 한국은행 주요 
업무를 이해하는데 필요한 전문 용어와 경제·금융 흐름을 이해하는데 도움이 되는 
시사 경제금융 용어들을 수록하였습니다. 용어해설은 개념과 도입 배경, 의미, 적용 
사례 등을 담아 쉽게 이해할 수 있도록 하였습니다. 또한 e-book으로도 제작하여 
독자들의 편의성과 가독성을 높였습니다 . 
코로나 19로 이전에 경험하지 못한 경제위기를 겪으면서 경제·금융 현상에 대한 
일반인들의 관심도가 이전에 비해 높아졌습니다. 아무쪼록 이 책자가 한국은행의 주요 
정책과 국내외 경제를 이해하는 데 조금이나마 도움이 되기를 바라겠습니다. 끝으로 
경제금융용어 700선
  집필에 참여해주신 경제교육실 교수님들과 용어 집필에 좋은 
의견을 주신 관련부서 여러분께 감사드립니다 .
2020년 8월
한국은행 경제교육실장 박 철 원
머 리 말


In [8]:

print(texts[5].page_content)

vi
경제금융용어 700선
ㅂ
바이오인증 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・118
바젤은행감독위원회/
바젤위원회(BCBS) ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・118
발행시장 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・119
발행중지화폐/유통정지화폐 ・・・・・・・・・・・・・・・120
방카슈랑스 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・120
배당할인모형 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・121
밴드웨건효과 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・121
뱅 크 런 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・122
범위의 경제 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・122
법률리스크 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・122
베블런효과 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・123
변동금리 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・123
변동금리부채권(FRN) ・ ・ ・・・・・・・・・・・・・・・・・・・・・・・123
보기화폐(견양화폐) ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・124
보완자본(Tier 2) ・・・・・・・・・・・

In [10]:
print(texts[12].page_content)

xiii
찾아보기❙
환매조건부매매/RP/Repo ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・331
환 어 음 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・332
환율조작국 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・332
환전영업자(환전상) ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・333
환차손/환차익 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・333
황금낙하산 ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・ ・334
회 사 채 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・334
회수의문 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・335
후방연쇄효과 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・335
후순위금융채 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・335
후행종합지수 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・336
ABC
B e y o n d  G D P ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・337
BIS 자기자본비율 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・338
CAMEL-R시스템/ROCA시스템/
C A C R E L 시 스 템 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・338
C L S 은 행 ・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・339
C M O ・・・・・・・・・・・・・・・・

In [11]:
print(texts[13].page_content)

1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙


In [9]:

texts = texts[13:]
print('줄어든 청크의 개수:', len(texts))

줄어든 청크의 개수: 353


In [10]:

print('첫번째 청크 출력 :', texts[0])


첫번째 청크 출력 : page_content='1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙' metadata={'producer': 'Smart PDF Imposition 1.0', 'creator': 'Smart PDF Imposition 1.0', 'creationdate': 'January 31, 2018 16:21', 'author': 'INSUNG DATA INC.', 'creationdate--text': 'January 31, 2018 16:21', 'moddate': 'January 31, 2018 16:21', 'moddate--text': 'January 31, 2018 16:21', 'subject': 'CoreImposition PDF 1.0 Output', 'ti

In [14]:
print('마지막 청크 출력 :', texts[-1])

마지막 청크 출력 : page_content='경제금융용어 700선
발행인 이주열
편집인 박철원
발행처 한국은행(www.bok.or.kr)
서울특별시 중구 세종대로 67(태평로 2가)
발행일 2020년 8월 3일
인  쇄 ㈜제일프린테크 
 본 자료는 한국은행 홈페이지(http://www.bok.or.kr>경제교육>온라인학습>일반인) 에서 
다운로드 받으실 수 있습니다.
 이 책자에 대한 문의는 한국은행 경제교육실 경제교육기획팀(02-759-5618)으로 연락 
하여 주시기 바랍니다.
 
 경제금융용어 700선
 은 
① 정부간행물판매센터(☎ 02-734-6818, http://www.gpcbooks.co.kr) 
② 한국경제서적(☎ 02-737-7498)
③ 경제서적(☎ 02-736-0640, http://kj-book.co.kr) 또는 주요 서점 등에서 매권당 
8,000원에 판매하고 있습니다.
ISBN 979-11-5538-393-3 03320' metadata={'producer': 'Smart PDF Imposition 1.0', 'creator': 'Smart PDF Imposition 1.0', 'creationdate': 'January 31, 2018 16:21', 'author': 'INSUNG DATA INC.', 'creationdate--text': 'January 31, 2018 16:21', 'moddate': 'January 31, 2018 16:21', 'moddate--text': 'January 31, 2018 16:21', 'subject': 'CoreImposition PDF 1.0 Output', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'source': '2020_경제금융용어 700선_게시.pdf', 'total_pages': 371, 'page': 369, 'page_label': '370'}


In [15]:

print('마지막에서 두번째 청크 출력 :', texts[-2])

마지막에서 두번째 청크 출력 : page_content='352
경제금융용어 700선
와 관련된 다양한 부수업무를 하는 회사를 말한다. 가령 신용카드 거래가 발생하면 
VAN사업자는 거래 승인을 위해 신용카드사에 거래내역을 전송하고 승인 및 조회결과를 
수신한 후 이를 가맹점에 전송한다. 또한 VAN사업자는 가맹점이 신용카드사에 전표를 
제출하여 대금을 청구하는 업무를 대신하기 위하여 가맹점을 방문하여 전표실물을 수거
하거나 전표 데이터를 수집한다 . 
 연관검색어 : 지급결제시스템, 전자금융
VaR(Value at Risk) 
주어진 신뢰수준 하에서 일정 기간 동안 발생할 수 있는 ‘최대 손실금액’으로 금융기관
의 잠재적인 손실을 측정하는 지표이다. 예를 들어 목표기간 1년, 신뢰수준 95% 에서 
산출된 VaR가 10억 원이라면 이는 1년 동안 발생할 수 있는 손실금액이 10억 원보다 
작을 확률이 95%라는 것을 의미한다 . 
 연관검색어 : 예상손실
VIX
미국 주식시장의 단기 변동성에 대한 시장의 기대치를 나타내는 지수로 시카고옵션거
래소(CBOE)에서 제공되며, 정식명칭은 CBOE Volatility Index이다. VIX는 향후 30 일 
동안의 S&P 500 지수의 변동성에 대한 시장의 기대치로서, 지수의 변동성이 클 것으로 
예상될 경우 옵션가격이 높아지는 점에 착안하여 CBOE에 상장된 다양한 행사가격의 
S&P 500 지수 옵션들의 가격을 활용하여 산출된다. VIX는 일반적으로 기초자산 가격과 
음(-)의 상관관계가 있다. 예를 들어 주가지수가 상승할 때 하락하고 주가지수가 하락할 
때는 상승한다. 이에 따라 VIX의 상승은 투자자들의 불안심리가 증대하는 것을 의미하므
로 공포지수라고도 부른다. 1993년 Robert E. Whaley 교수의 논문에서 처음 소개된 
이후 시장의 변동성과 투자자들의 심리를 나타내주는 주요한 지표로 널리 활용되고 
있다. 이 외의 대표적인 주식시장 변동성 지수로는 유럽의 VSTOXX 등이 있다. VSTOXX
는

In [12]:
texts = texts[:-1]
print('마지막 데이터 제거 후 청크의 개수:', len(texts))

마지막 데이터 제거 후 청크의 개수: 352


In [13]:
embedding = OpenAIEmbeddings(chunk_size=100)

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding)

In [14]:
print(vectordb._collection.count())

705


In [15]:

for key in vectordb._collection.get():
  print(key)

ids
embeddings
documents
uris
included
data
metadatas


In [16]:

documents = vectordb._collection.get()['documents']
print('청크의 개수 :', len(documents))
print('-' * 50)
print('0번 청크 출력 :', documents[0])

청크의 개수 : 705
--------------------------------------------------
0번 청크 출력 : 1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙


In [17]:
embeddings = vectordb._collection.get(include=['embeddings'])['embeddings']
print('임베딩 벡터의 개수 :', len(embeddings))

임베딩 벡터의 개수 : 705


In [22]:

print('0번 청크의 임베딩 값 출력 :', embeddings[0])
print('0번 청크의 임베딩 값의 길이 :', len(embeddings[0]))

0번 청크의 임베딩 값 출력 : [-0.00759038 -0.00721978  0.01452192 ... -0.01861221 -0.00679428
 -0.0088257 ]
0번 청크의 임베딩 값의 길이 : 1536


In [18]:
metadatas = vectordb._collection.get()['metadatas']
print('metadatas의 개수 :', len(metadatas))
print('0번 청크의 출처 :', metadatas[0])

metadatas의 개수 : 705
0번 청크의 출처 : {'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'creationdate': 'January 31, 2018 16:21', 'producer': 'Smart PDF Imposition 1.0', 'subject': 'CoreImposition PDF 1.0 Output', 'page_label': '18', 'moddate': 'January 31, 2018 16:21', 'source': '2020_경제금융용어 700선_게시.pdf', 'moddate--text': 'January 31, 2018 16:21', 'total_pages': 371, 'page': 17, 'creationdate--text': 'January 31, 2018 16:21', 'author': 'INSUNG DATA INC.', 'creator': 'Smart PDF Imposition 1.0'}


In [48]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

docs = retriever.get_relevant_documents("가상통화 궁금해")
print('유사 문서 개수 :', len(docs))
print('--' * 20)
print('첫번째 유사 문서 :', docs[0])
print('두번째 유사 문서 :', docs[1])

유사 문서 개수 : 2
----------------------------------------
첫번째 유사 문서 : page_content='72
경제금융용어 700선
기축통화
기축통화란 여러 국가의 암묵적인 동의하에 국제거래에서 중심적인 역할을 하는 
통화를 지칭한다. 구체적으로는 ① 국제무역결제에 사용되는 통화 ② 환율 평가 시의 
지표가 되는 통화 ③ 대외준비자산으로 보유되는 통화 등의 의미를 포함한다. 그러므로 
어떤 나라의 통화가 기축통화가 되기 위해서는 세계적으로 원활히 유통될 수 있도록 
유동성이 풍부하여야 하고 거래당사자들이 믿고 사용할 수 있도록 신뢰성을 갖추어야 
하며, 국제적으로 경제력은 물론 정치력･군사력까지 인정받는 국가의 통화여야 한다. 
20세기 초반까지는 세계 금융경제의 중심이었던 영국의 파운드화가 기축통화로서 국제
거래에 주로 이용되었으며, 2차 세계대전 이후에는 전 세계 외환거래 및 외환보유액의 
상당 부분을 차지하는 미국 달러화가 기축통화로 인정받고 있다. 한편, 현재 국제거래에
서 비교적 자주 사용되는 통화로는 미국 달러화이외에도 유로화, 영국 파운드화, 일본 
엔화, 스위스 프랑화 등이 있는데 이들은 흔히 교환성통화라 한다 . 
 연관검색어 : 교환성 통화
기타기본자본(Additional Tier 1)
기타기본자본(AT1; Additional Tier 1)은 요건을 충족하는 기타기본자본증권 및 이와 
관련된 자본잉여금, 자회사가 발행한 기타기본자본증권의 제3자 보유분 중 일부 금액 
등으로 구성된다. 기타기본자본증권으로 인정받기 위해서는 예금자, 일반채권자 및 
후순위채권보다 후순위, 영구적, 배당 및 이자지급의 임의성, 중도상환 유인이 없을 
것 등 총 14개 요건을 충족해야 한다 .
기펜재
일반적으로 재화는 수요의 법칙에 따라 가격이 하락하면 수요량이 늘어난다. 그러나 
예외적으로 재화의 가격이 하락할 때 수요량이 감소하는 현상을 관찰할 수 있다. 이렇게 
가격이 하락할 경우 수요의 법칙에 위배되는 재화를 이를 처음 

In [20]:

template = """당신은 한국은행에서 만든 금융 용어를 설명해주는 금융해설자입니다.
주어진 검색 결과를 바탕으로 답변하세요.
검색 결과에 없는 내용이라면 답변할 수 없다고 하세요. 반말로 친근하게 답변하세요.
{context}

Question: {question}
Answer:
"""

prompt = PromptTemplate.from_template(template)

In [21]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [49]:
# ============================
# 5. Ensemble Retriever (BM25+FAISS)
# ============================
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever # Corrected import

vector_retriever = vectordb.as_retriever(search_kwargs={"k": 3})

bm25 = BM25Retriever.from_documents(texts)
bm25.k = 3

ensemble_retriever = EnsembleRetriever(retrievers=[bm25, vector_retriever], weights=[0.4, 0.6])

docs = ensemble_retriever.get_relevant_documents("인플레이션이 무엇인가요?")
print("\n🔹 Ensemble Retriever 결과:")
for d in docs:
    print("-", d.page_content[:120].replace("\n"," ",), d.metadata)


🔹 Ensemble Retriever 결과:
- 225 ㅇ  인터넷전문은행 점포없이 또는 소수의 영업점만을 두고 인터넷, 모바일, ATM 등 전자매체를 주된  영업채널로 활용하는 온라인 기반의 은행으로서, 이는 은행서비스를 인터넷으로 제공하 는 영업방식을 뜻하는  {'author': 'INSUNG DATA INC.', 'subject': 'CoreImposition PDF 1.0 Output', 'page_label': '242', 'total_pages': 371, 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'creationdate--text': 'January 31, 2018 16:21', 'producer': 'Smart PDF Imposition 1.0', 'moddate': 'January 31, 2018 16:21', 'creationdate': 'January 31, 2018 16:21', 'source': '2020_경제금융용어 700선_게시.pdf', 'moddate--text': 'January 31, 2018 16:21', 'page': 241, 'creator': 'Smart PDF Imposition 1.0'}
- 66 경제금융용어 700선 핵심지표중 하나이다. 기대인플레이션은 임금협상, 가격설정, 투자결정 등 경제주체의  의사결정에 반영되면서 최종적으로 실제 인플레이션에 영향을 미친다. 구체적인 경로를  살펴보면 기대인플레이 {'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'moddate--text': 'January 31, 2018 16:21', 'page': 82, 'total_pages': 371, 'creator': 'Smart PDF Imposition 1.0', 'page_label': '83', 'producer': 'Smart PDF Imposition 1.0', 'creationdate--text': 'January 31, 2018 16:21', 'source': 

In [ ]:

from langchain_core.tracers.stdout import ConsoleCallbackHandler


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type_kwargs={"prompt": prompt},
    retriever=ensemble_retriever,
    return_source_documents=True)

In [51]:

input_text = "virtual currency 무엇인가?"


docs = ensemble_retriever.get_relevant_documents(input_text)
print("\n🔹 Ensemble Retriever 결과:")
for d in docs:
    print("-", d.page_content[:120].replace("\n"," ",), d.metadata)

chatbot_response = qa_chain(input_text)
print(chatbot_response)    


🔹 Ensemble Retriever 결과:
- 250 경제금융용어 700선 전자화폐 IC카드 등에 화폐가치를 저장하였다가 상품 등의 구매에 사용할 수 있는 전자지급수단 으로서 범용성과 환금성을 갖춘 것을 말한다. 전자화폐(electronic money)의 유형은 {'creationdate--text': 'January 31, 2018 16:21', 'moddate--text': 'January 31, 2018 16:21', 'producer': 'Smart PDF Imposition 1.0', 'moddate': 'January 31, 2018 16:21', 'page_label': '267', 'creator': 'Smart PDF Imposition 1.0', 'author': 'INSUNG DATA INC.', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'total_pages': 371, 'creationdate': 'January 31, 2018 16:21', 'source': '2020_경제금융용어 700선_게시.pdf', 'page': 266, 'subject': 'CoreImposition PDF 1.0 Output'}
- 6 경제금융용어 700선 가상통화공개(ICO)  가상통화(ICO; Initial Coin Offering) 공개는 주로 혁신적인 신생기업(startup)이 암호 화화폐(cryptocurrency) 또는 디지털 토큰( {'subject': 'CoreImposition PDF 1.0 Output', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'creator': 'Smart PDF Imposition 1.0', 'page': 22, 'source': '2020_경제금융용어 700선_게시.pdf', 'moddate': 'January 31, 2018 16:21', 'page_label': '23', 'moddate--text': 'January 31, 2018 16:21', 

In [33]:
def get_chatbot_response(input_text):
    chatbot_response = qa_chain(input_text)
    return chatbot_response['result'].strip()

In [34]:
input_text = "너는 뭘하는 챗봇이니?"
result = get_chatbot_response(input_text)
print(result)

미안하지만, 너의 질문에 대한 답변은 제공할 수 없어. 다른 금융 용어에 대해 궁금한 게 있으면 물어봐!


In [37]:
input_text = "가상통화 대해서 궁금하당~"
result = get_chatbot_response(input_text)
print(result)

가상통화는 중앙은행이나 금융기관이 아닌 민간에서 블록체인 기술을 기반으로 발행되고 유통되는 '가치의 전자적 표시'야. 비트코인이 가장 대표적인 가상통화로, 중앙운영기관 없이 P2P 거래가 가능한 분산형 시스템을 통해 발행되고 유통돼. 기존의 사이버머니나 멤버십 포인트와는 다르게 중앙에서 발행이나 유통을 통제하지 않는다는 점이 특징이지. 최근에는 비트코인 외에도 다양한 신종코인들이 등장하면서, 이들을 암호통화라고 부르기도 해.


In [42]:
# 인터페이스 생성
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="경제금융용어 챗봇") # 챗봇 레이블을 좌측 상단에 구성
    msg = gr.Textbox(label="질문해주세요!")  # 하단의 채팅창 레이블
    clear = gr.Button("대화 초기화")  # 대화 초기화 버튼

    # 챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
      result = qa_chain(message, 
                        callbacks=[ConsoleCallbackHandler()])
      bot_message = result['result']

      # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가
      chat_history.append((message, bot_message))
      return "", chat_history

    # 사용자의 입력을 제출(submit)하면 respond 함수가 호출
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화
    clear.click(lambda: None, None, chatbot, queue=False)

# 인터페이스 실행
demo.launch(debug=True)

/var/folders/xn/w4kqb1715fz7h49tp_h7hbmr0000gn/T/ipykernel_12774/1335127311.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="경제금융용어 챗봇") # 챗봇 레이블을 좌측 상단에 구성


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "가상통화란"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "가상통화란",
  "context": "72\n경제금융용어 700선\n기축통화\n기축통화란 여러 국가의 암묵적인 동의하에 국제거래에서 중심적인 역할을 하는 \n통화를 지칭한다. 구체적으로는 ① 국제무역결제에 사용되는 통화 ② 환율 평가 시의 \n지표가 되는 통화 ③ 대외준비자산으로 보유되는 통화 등의 의미를 포함한다. 그러므로 \n어떤 나라의 통화가 기축통화가 되기 위해서는 세계적으로 원활히 유통될 수 있도록 \n유동성이 풍부하여야 하고 거래당사자들이 믿고 사용할 수 있도록 신뢰성을 갖추어야 \n하며, 국제적으로 경제력은 물론 정치력･군사력까지 인정받는 국가의 통화여야 한다. \n20세기 초반까지는 세계 금융경제의 중심이었던 영국의 파운드화가 기축통화로서 국제\n거래에 주로 이용되었으며, 2차 세계대전 이후에는 전 세계 외환거래 및 외환보유액의 \n상당 부분을 차지하는 미국 달러화가 기축통화로 인정받고 있다. 한편, 현재 국제거래에\n서 비교적 자주 사용되는 통화로는 미국 달러화이외에도 유로화, 영국 파운드화, 일본 \n엔화, 스위스 프랑화 등이 있는데 이들은 흔히 교환성통화라 한다 . \n 연관검색어 : 교환성 통화\n기타기본자본(Additional Tier 1)\n기타기본자본(AT1; Additional Tier 1)은 요건을 충족하는 기타기본자본증권 및 이와 \n관련된 자본잉여금, 자회사가 발행한 

In [44]:
demo.close()